In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2

In [2]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

In [22]:
from src.similarities import Embedding, Similarities
from src.keywords import Keyword, Keywords

In [23]:
from src.keywords import Keywords

In [6]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. 
# Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=2,
    hf=True
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/mac/miniforge3/envs/phd/lib/python3.8/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
from transformers import AutoTokenizer, pipeline

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [8]:
response = generator("What is 1+1?")
print(response[0]["generated_text"])

What is 1+1?
A: 2


In [9]:
prompt = """
I have the following document:
* The website mentions that it only takes a couple of days to deliver but I still have not received mine

Extract 5 keywords from that document.
"""
response = generator(prompt)
print(response[0]["generated_text"])


I have the following document:
* The website mentions that it only takes a couple of days to deliver but I still have not received mine

Extract 5 keywords from that document.

**Answer:**
1. Website
2. Deliver
3. Couple
4. Days
5. Mention


In [10]:
example_prompt = """
<s>[INST]
I have the following document:
- The website mentions that it only takes a couple of days to deliver but I still have not received mine.

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST] meat, beef, eat, eating, emissions, steak, food, health, processed, chicken</s>"""

In [11]:
keyword_prompt = """
[INST]
I have the following document:
- [DOCUMENT]

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST]
"""

In [12]:
prompt = example_prompt + keyword_prompt

In [13]:
from keybert.llm import TextGeneration
from keybert import KeyLLM

# Load it in KeyLLM
llm = TextGeneration(generator, prompt=prompt)
kw_model = KeyLLM(llm)

In [29]:
documents = [
"The website mentions that it only takes a couple of days to deliver but I still have not received mine.",
"I received my package!",
"Whereas the most powerful LLMs have generally been accessible only through limited APIs (if at all), Meta released LLaMA's model weights to the research community under a noncommercial license."
]

keywords = kw_model.extract_keywords(documents)

In [28]:
similarities = Similarities('bert-base-uncased')
embedding = Embedding(similarities.model, similarities.tokenizer)

keywords_helper = Keywords(similarities.model, similarities.tokenizer)
keywords_helper.get_word_embedding('this is a sentence and a sentence', 'sentence')

No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.


False

In [32]:
keywords_helper.get_word_embedding(documents[0], keywords[0][1])

False

In [19]:
from keybert import KeyLLM
from sentence_transformers import SentenceTransformer

# Extract embeddings
model = SentenceTransformer('BAAI/bge-small-en-v1.5')
embeddings = model.encode(documents, convert_to_tensor=True)

# Load it in KeyLLM
kw_model = KeyLLM(llm)

# Extract keywords
keywords, _ = kw_model.extract_keywords(
    documents, 
    embeddings=embeddings, 
    threshold=.5
)

ValueError: too many values to unpack (expected 2)

In [17]:
keywords

[['website',
  'mention',
  'days',
  'deliver',
  'couple',
  'receive',
  'mine',
  'still',
  'not',
  'yet'],
 ['website',
  'mention',
  'days',
  'deliver',
  'couple',
  'receive',
  'mine',
  'still',
  'not',
  'yet'],
 ['LLM',
  'API',
  'accessibility',
  'release',
  'license',
  'research',
  'community',
  'model',
  'weights',
  'Meta',
  'power',
  'availability',
  'commercial',
  'noncommercial',
  'language',
  'models',
  'development',
  'collaboration',
  'openness',
  'sharing',
  'innovation',
  'technology',
  'artificial']]

In [18]:
from keybert import KeyLLM, KeyBERT

# Load it in KeyLLM
kw_model = KeyBERT(llm=llm, model='BAAI/bge-small-en-v1.5')

# Extract keywords
keywords = kw_model.extract_keywords(documents, threshold=0.5)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
keywords